# Amazon-1

In [38]:
from flask import Flask, render_template, request,jsonify
from flask_cors import CORS,cross_origin
import requests
from requests import Session 
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq
import pandas as pd


In [39]:
amazon_url = "https://www.amazon.in/s?k="+"book"
uClient = uReq(amazon_url)
uClient;
amazonPage= uClient.read()
amazonPage;
uClient.close()

In [40]:
amazon_html = bs(amazonPage,'html.parser')
amazon_html;

# Home Page for search results

#bigboxes = amazon_html.findAll("div", {"class":"s-card-container s-overflow-hidden aok-relative puis-include-content-margin puis s-latency-cf-section s-card-border"})

bigboxes = amazon_html.findAll('div', attrs={'data-component-type':'s-search-result'})
bigboxes;

In [41]:
headers = { 
			"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
			"accept-encoding": "gzip, deflate, br", 
			"accept-language": "en", 
			"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36", 
		}

session = Session()
session.headers= headers

In [42]:


def get_product_review_link(bigboxes):
    all_review_page_list = []
    product_id_list = []
    product_name_list=[]
    for box in bigboxes:
        #link_tag = box.find("a", attrs={'class':'a-link-normal s-no-outline'})
        productLink = "https://www.amazon.in"+ box.div.div.div.div.a["href"]
        index = productLink.find('dp/')
        see_all_review_page = productLink[:index] + "product-reviews" + productLink[index+2:]
        all_review_page_list.append(see_all_review_page)
        #links_tag_list.append("https://www.amazon.in"+ box.div.div.div.div.a["href"])
        product_id_list.append(box.get("data-asin"))
        product_name_list.append(box.div.h2.span)
        
    return product_id_list, product_name_list, all_review_page_list


In [43]:
product_id, product_name, all_review_page_list = get_product_review_link(bigboxes)

In [44]:

def get_reviews(all_review_page_list,product_id,review_page_number_range):
    
    a = review_page_number_range[0]
    b = review_page_number_range[1]
    
    headers = { 
                "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
                "accept-encoding": "gzip, deflate, br", 
                "accept-language": "en", 
                "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36", 
            }

    #session = Session()
    #session.headers= headers    
    customer_name_list = []
    rating_list = []
    review_title_list = []
    reviews_list = []
    product_ID_final = []
    
    for link in all_review_page_list:
        #print('productLink is: ',link) 
        index_for_product_id = iter([i for i in range(len(all_review_page_list))])
        productID = product_id[next(index_for_product_id)]
        
        for review_page_number in range(a,b):
            next_review_page_link = link + f"/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber={review_page_number}&reviewerType=all_reviews"
            #print(next_review_page)    
            next_review_page = requests.get(next_review_page_link, headers=headers)
            next_review_page.encoding='utf-8'
            next_review_page_soup = bs(next_review_page.content, "html.parser")
            profiles = next_review_page_soup.findAll("span", {"class":"a-profile-name"})
            #print(next_review_page_link)
            for i in range(len(profiles)):
                customer_name = profiles[i].text

                Review_title = next_review_page_soup.findAll('span',{'class':"a-size-base review-title-content a-color-base a-text-bold",'data-hook':"review-title"})[i].text
                reviews = next_review_page_soup.findAll("span", {"data-hook":"review-body"})[i].text
                rating = next_review_page_soup.findAll("span", {"class":"a-icon-alt"})[i].text
                #print(next_review_page_link)
                #print(product_name)
                product_ID_final.append(productID)
                customer_name_list.append(customer_name)
                rating_list.append(rating)
                review_title_list.append(Review_title)
                reviews_list.append(reviews)
    df = pd.DataFrame([product_ID_final,customer_name_list, rating_list, review_title_list, reviews_list]).T
    df.columns = ["Product ID",'Customer Name','Rating','Review Title','Reviews']
    df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])
    df['Review Title'] = df['Review Title'].apply(lambda x: x.strip())
    df['Reviews'] = df['Reviews'].apply(lambda x: x.strip())
    return df


In [45]:
df1 = get_reviews(all_review_page_list,product_id,[1,2])

In [46]:
df1

,Product ID,Customer Name,Rating,Review Title,Reviews
0,8172345666,Arjun Singh,4.6,The boy who lived,It's just awesome. everything are in new and g...
1,8172345666,Aishwarya Rathore,5.0,Everybody received book marks I didn’t 😓,
2,8172345666,Sriranga,4.0,Awesome,Took me back to my childhood. I started my jou...
3,8172345666,Laxmi,5.0,Fast delivery and books in good condition.. Wi...,"Good packaging, Bookmarks and packageBooks tot..."
4,8172345666,D Dalal,5.0,Good but bookmarks missing,Bookmarks are missing
...,...,...,...,...,...
85,8172345666,R.P. Mishra,5.0,Good for early,Got it for my little sister she likes to read....
86,8172345666,Ashfaq,5.0,Best for kids,Good for kids learning
87,8172345666,Amazon Customer,4.0,Identifying the things & observation skills,This product is one of the best learning pract...
88,8172345666,Ravi gupta,5.0,Best,This product is one of the best learning pract...


In [ ]:
# TEST TEST


def get_reviews(all_review_page_list,product_id,review_page_number_range,product_pageNo):
    
    a = review_page_number_range[0]
    b = review_page_number_range[1]
    
    headers = { 
                "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
                "accept-encoding": "gzip, deflate, br", 
                "accept-language": "en", 
                "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36", 
            }

    #session = Session()
    #session.headers= headers    
    customer_name_list = []
    rating_list = []
    review_title_list = []
    reviews_list = []
    product_ID_final = []
    for page in range(1,product_pageNo):
        for link in all_review_page_list:
            #print('productLink is: ',link) 
            index_for_product_id = iter([i for i in range(len(all_review_page_list))])
            productID = product_id[next(index_for_product_id)]

            for review_page_number in range(a,b):
                next_review_page_link = link + f"/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber={review_page_number}&reviewerType=all_reviews"
                #print(next_review_page)    
                next_review_page = requests.get(next_review_page_link, headers=headers)
                next_review_page.encoding='utf-8'
                next_review_page_soup = bs(next_review_page.content, "html.parser")
                profiles = next_review_page_soup.findAll("span", {"class":"a-profile-name"})
                #print(next_review_page_link)
                for i in range(len(profiles)):
                    customer_name = profiles[i].text

                    Review_title = next_review_page_soup.findAll('span',{'class':"a-size-base review-title-content a-color-base a-text-bold",'data-hook':"review-title"})[i].text
                    reviews = next_review_page_soup.findAll("span", {"data-hook":"review-body"})[i].text
                    rating = next_review_page_soup.findAll("span", {"class":"a-icon-alt"})[i].text
                    #print(next_review_page_link)
                    #print(product_name)
                    product_ID_final.append(productID)
                    customer_name_list.append(customer_name)
                    rating_list.append(rating)
                    review_title_list.append(Review_title)
                    reviews_list.append(reviews)
    df = pd.DataFrame([product_ID_final,customer_name_list, rating_list, review_title_list, reviews_list]).T
    df.columns = ["Product ID",'Customer Name','Rating','Review Title','Reviews']
    df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])
    df['Review Title'] = df['Review Title'].apply(lambda x: x.strip())
    df['Reviews'] = df['Reviews'].apply(lambda x: x.strip())
    return df


# Amazon -2

In [ ]:
from flask import Flask, render_template, request,jsonify
from flask_cors import CORS,cross_origin
import requests
from requests import Session 
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq

#################################
import pandas as pd
import re
import time

In [ ]:

def search_product(search):
    count=0  # count determine number of iteration in case except condition becomes true. Loop will broke ou as soon as first condition become true
    while count<=50:
        try:
            amazon_url = "https://www.amazon.in/s?k="+ search
            uClient = uReq(amazon_url)
            amazonPage= uClient.read()
            uClient.close()
            #amazon_html = bs(amazonPage,'html.parser')
            #bigboxes = amazon_html.findAll("div", {"class":"s-card-container s-overflow-hidden aok-relative puis-include-content-margin puis s-latency-cf-section s-card-border"})
            print('Connection Established')
            break
            
        except Exception as e:
            print('Retrying: Check Your Internet Connection:',e)
            #continue
            time.sleep(2)
        count+=1
            
            
    return amazon_url        
        

In [ ]:
amazon_url = search_product("iphone")

In [ ]:
#no_pages = 10

def get_data(pageNo,amazon_url):
    headers = { 
			"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
			"accept-encoding": "gzip, deflate, br", 
			"accept-language": "en-GB,en-US;q=0.9,en;q=0.8", 
			"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36", 
		}

    
    #headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    #r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    count = 0
    while count<=60:
        try:
            r = requests.get(amazon_url+'&page='+str(pageNo), headers=headers)#, proxies=proxies)
            search_result = r.content
            break
        except Exception as e:
            print("Retrying...",e)
            time.sleep(1)
        count +=1
    
    r.encoding='utf-8'
    #search_result = r.content
    soup = bs(search_result,'html.parser')
    #print(soup)
    #results = soup.findAll('div', attrs={'data-component-type':'s-search-result'})
    #product_id.append(box.get("data-asin"))
    product_id = []
    alls = []
    
    for d in soup.findAll('div', attrs={'data-component-type':'s-search-result'}):
        #print(d)
        product_id.append(d.get("data-asin"))
        name = d.div.h2.span
        rating = d.find('span',{'class':'a-icon-alt'})
        customer_rated = d.find('span',{"class":"a-size-mini a-color-base puis-light-weight-text"})
        discounted_price= d.find('span',{'class':'a-offscreen'})
        price = d.find('div',{'class':"a-section a-spacing-none a-spacing-top-micro s-price-instructions-style"})
        discount = d.find("div",attrs={"class":"a-color-base"}).findAll("span",{"class":"puis-light-weight-text"})
        #print(discount)
        
        all1=[]
        if name is not None:
            all1.append(name.text)
            #print(name)
        else:
            all1.append("unknown-product")

        if rating is not None:
            #print(rating.text)
            all1.append(rating.text)
        else:
            all1.append('NA')

        if customer_rated is not None:
            #print(price.text)
            all1.append(customer_rated.text)
        else:
            all1.append('0')

        if discounted_price != []:
            #print(price.text)
            try:
                all1.append(discounted_price.text)
            except:
                return "Scrapping Error In Discounted Price"
        else:
            all1.append('NA')

        if discount != []:
            #print(price.text)
            try:
                all1.append(discount[1].text)
            except:
                all1.append(discount[0].text)
                pass
        else:
            all1.append('NA')

        if price is not None:
            #print(price.text)
            try:
                all1.append(price.find('span',{'class':'a-text-price'}).span.text)
                #print(price.div.find('span',{'class':'a-text-price'}).span.text)
                
            except Exception as e:
                
                #return " Something Is Wrong In Scrapping Price"
                all1.append(price.find('span',{'class':'a-text-price'}))
                pass
        else:
            all1.append('NA')
            
            
            
        alls.append(all1)    
            
            
    return alls, product_id

#get_data(1,amazon_url)

In [ ]:
get_data(1,amazon_url)

In [ ]:
    pageNo=1
    headers = { 
			"accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
			"accept-encoding": "gzip, deflate, br", 
			"accept-language": "en-GB,en-US;q=0.9,en;q=0.8", 
			"user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36", 
		}

    
    #headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}

    #r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)
    count = 0
    while count<=40:
        try:
            r = requests.get(amazon_url+'&page='+str(pageNo), headers=headers)#, proxies=proxies)
            search_result = r.content
            break
        except Exception as e:
            print("Retrying...",e)
            time.sleep(1)
        count +=1
    
    r.encoding='utf-8'
    #search_result = r.content
    soup = bs(search_result,'html.parser')
    #print(soup)
    #results = soup.findAll('div', attrs={'data-component-type':'s-search-result'})
    #product_id.append(box.get("data-asin"))
    product_id = []
    alls = []
    
    for d in soup.findAll('div', attrs={'data-component-type':'s-search-result'}):
        #print(d)
        product_id.append(d.get("data-asin"))
        name = d.div.h2.span
        rating = d.find('span',{'class':'a-icon-alt'})
        customer_rated = d.find('span',{"class":"a-size-mini a-color-base puis-light-weight-text"})
        discounted_price= d.find('span',{'class':'a-offscreen'})
        price = d.find('div',{'class':"a-section a-spacing-none a-spacing-top-micro s-price-instructions-style"})
        discount = d.find("div",attrs={"class":"a-color-base"}).findAll("span",{"class":"puis-light-weight-text"})
        print(price.find('span',{'class':'a-text-price'}))

In [ ]:
# Iterate over number of pages and fetch and proces the data to dataframe
#no_pages=40
# for loop for limiting the the number of times of retrials
#incase of some problemm in executoing of following code


def generate_DataFrame(number_of_pages,amazon_url):
    count = 0 
    while count<=30:
        try:
            no_pages= number_of_pages
            results = []
            results_all_other = []
            results_product_id=[]
            for i in range(1, no_pages+1):
                all_other, product_id = get_data(i,amazon_url)
                results_all_other.append(all_other)
                results_product_id.append(product_id)

                flatten = lambda l: [item for sublist in l for item in sublist]
                df = pd.DataFrame(flatten(results_all_other),columns=['Product Name','Rating','Customers Rated','Dicounted Price', 'Discount','Price'])
                df.insert(0,"Product ID",flatten(results_product_id),allow_duplicates=True)
                # Data Processing
                break
        except:
            print("Retrying...")
            #continue
            #time.sleep(3) # sleep time for execution of time
        count +=1
        
    df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])
    #df["Price"] = df["Price"].str.replace('₹', '',regex=True)
    
    df["Price"] = df["Price"].str.replace(',', '',regex=True)
    df["Customers Rated"] = df["Customers Rated"].str.replace('(', '',regex=True)
    df["Customers Rated"] = df["Customers Rated"].str.replace(')', '',regex=True)
    df["Discount"] = df["Discount"].str.replace('(', '',regex=True)
    df["Discount"] = df["Discount"].str.replace(')', '',regex=True)
    df["Discount"] = df["Discount"].str.replace(' off', '',regex=True)
    
    df.to_csv('amazon_products.csv', index=False, encoding='utf-8') # Export Dataframe to csv
    #df

    return df

In [ ]:
generate_DataFrame(4,amazon_url)

# Combined

In [10]:
from flask import Flask, render_template, request,jsonify
from flask_cors import CORS,cross_origin
import requests
from requests import Session 
from bs4 import BeautifulSoup as bs
from urllib.request import urlopen as uReq

#################################
import pandas as pd
import re
import time
import random

In [11]:
class Amazon:
    
    ############################ Function For Searching the product ################################
    
    def search_product(search):
        count=0  # count determine number of iteration in case except condition becomes true. Loop will broke ou as soon as first condition become true
        while count<=50:
            try:
                amazon_url = "https://www.amazon.in/s?k="+ search
                uClient = uReq(amazon_url)
                amazonPage= uClient.read()
                uClient.close()
                #amazon_html = bs(amazonPage,'html.parser')
                #bigboxes = amazon_html.findAll("div", {"class":"s-card-container s-overflow-hidden aok-relative puis-include-content-margin puis s-latency-cf-section s-card-border"})
                print('Connection Established')
                break
            except Exception as e:
                print('Retrying: Check Your Internet Connection:',e)
                #continue
                time.sleep(2)
            count+=1

        return amazon_url        
    #no_pages = 10
    
    ###################### Function for get product informaion displayed on search page ###########################

    def get_products_data(pageNo,amazon_url):
        headers = { 
                "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
                "accept-encoding": "gzip, deflate, br", 
                "accept-language": "en-GB,en-US;q=0.9,en;q=0.8", 
                "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36", 
            }

        #headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64; rv:66.0) Gecko/20100101 Firefox/66.0", "Accept-Encoding":"gzip, deflate", "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
        #r = requests.get('https://www.amazon.in/gp/bestsellers/books/ref=zg_bs_pg_'+str(pageNo)+'?ie=UTF8&pg='+str(pageNo), headers=headers)#, proxies=proxies)

        count = 0
        while count<=60:
            try:
                r = requests.get(amazon_url+'&page='+str(pageNo), headers=headers)#, proxies=proxies)
                search_result = r.content
                break
            except Exception as e:
                print("Retrying...",e)
                time.sleep(1)
            count +=1

        r.encoding='utf-8'
        #search_result = r.content
        soup = bs(search_result,'html.parser')
        #print(soup)
        #results = soup.findAll('div', attrs={'data-component-type':'s-search-result'})
        #product_id.append(box.get("data-asin"))
        product_id = []
        alls = []

        for d in soup.findAll('div', attrs={'data-component-type':'s-search-result'}):
            #print(d)
            product_id.append(d.get("data-asin"))
            name = d.div.h2.span
            rating = d.find('span',{'class':'a-icon-alt'})
            customer_rated = d.find('span',{"class":"a-size-mini a-color-base puis-light-weight-text"})
            discounted_price= d.find('span',{'class':'a-offscreen'})
            price = d.find('div',{'class':"a-section a-spacing-none a-spacing-top-micro s-price-instructions-style"})
            discount = d.find("div",attrs={"class":"a-color-base"}).findAll("span",{"class":"puis-light-weight-text"})
            #print(discount)

            all1=[]
            if name is not None:
                all1.append(name.text)
                #print(name)
            else:
                all1.append("unknown-product")

            if rating is not None:
                #print(rating.text)
                all1.append(rating.text)
            else:
                all1.append('NA')

            if customer_rated is not None:
                #print(price.text)
                all1.append(customer_rated.text)
            else:
                all1.append('0')

            if discounted_price != []:
                #print(price.text)
                try:
                    all1.append(discounted_price.text)
                except:
                    return "Scrapping Error In Discounted Price"
            else:
                all1.append('NA')

            if discount != []:
                #print(price.text)
                try:
                    all1.append(discount[1].text)
                except:
                    all1.append(discount[0].text)
                    pass
            else:
                all1.append('NA')

            if price is not None:
                #print(price.text)
                try:
                    all1.append(price.find('span',{'class':'a-text-price'}).span.text)
                    #print(price.div.find('span',{'class':'a-text-price'}).span.text)

                except Exception as e:

                    #return " Something Is Wrong In Scrapping Price"
                    all1.append(price.find('span',{'class':'a-text-price'}))
                    pass
            else:
                all1.append('NA')
            alls.append(all1)    

        return alls, product_id

    #get_data(1,amazon_url)       

    ############## Function For Generating The Dataframe For Details Extracted from Search Page ####################
    
    def generate_detailed_df(number_of_pages_range,amazon_url):
        start_page = number_of_pages_range[0]
        end_page = number_of_pages_range[1]
        
        #no_pages= number_of_pages
        results = []
        results_all_other = []
        results_product_id=[]
       
        for i in range(start_page, end_page+1):
            count = 0 
            while count<=30:
                try:
                    print(f"Scrapping Page {i} of search results")
                    all_other, product_id = Amazon.get_products_data(i,amazon_url)
                    results_all_other.append(all_other)
                    results_product_id.append(product_id)
                    #print(f"Page {i} scrapped")
                    #print(len(results_all_other))
                    if (len(all_other)!=0 & len(product_id)!=0):
                        print(f"Page {i} scrapped")
                    else:
                        print("Incomplete Scrapping: Few or Many Product May have been missed")
                        print("We recommend you to retry after some time")
                    
                    #print(product_id)
                    #flatten = lambda l: [item for sublist in l for item in sublist]
                    #df = pd.DataFrame(flatten(results_all_other),columns=['Product Name','Rating','Customers Rated','Dicounted Price', 'Discount','Price'])
                    #df.insert(0,"Product ID",flatten(results_product_id),allow_duplicates=True)
                    # Data Processing
            
                    break
                except:
                    print("Retrying...")
                    #continue
                    #time.sleep(3) # sleep time for execution of time
                count +=1
        
        flatten = lambda l: [item for sublist in l for item in sublist]
        df = pd.DataFrame(flatten(results_all_other),columns=['Product Name','Rating','Customers Rated','Dicounted Price', 'Discount','Price'])
        df.insert(0,"Product ID",flatten(results_product_id),allow_duplicates=True)
          
        df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])
        #df["Price"] = df["Price"].str.replace('₹', '',regex=True)

        df["Price"] = df["Price"].str.replace(',', '',regex=True)
        df["Price"] = df["Price"].str.replace('₹', '',regex=True)
        df["Customers Rated"] = df["Customers Rated"].str.replace('(', '',regex=True)
        df["Customers Rated"] = df["Customers Rated"].str.replace(')', '',regex=True)
        df["Discount"] = df["Discount"].str.replace('(', '',regex=True)
        df["Discount"] = df["Discount"].str.replace(')', '',regex=True)
        df["Discount"] = df["Discount"].str.replace(' off', '',regex=True)
        df.to_csv('amazon_products.csv', index=False, encoding='utf-8') # Export Dataframe to csv

        return df
    
    ################### function for Getting list of All Major Links like all_review page,product_id, and product name list ########
    
    def get_products_review_link(amazon_url,pageNo_range):
        start_page = pageNo_range[0]
        end_page = pageNo_range[1]

        #headers = { 
        #            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
        #            "accept-encoding": "gzip, deflate, br", 
        #            "accept-language": "en", 
        #            "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36", 
        #        }
        headers = { 
                "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
                "accept-encoding": "gzip, deflate, br", 
                "accept-language": "en-GB,en-US;q=0.9,en;q=0.8", 
                "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36", 
            }
        
        #r = requests.get(amazon_url+'&page='+str(pageNo), headers=headers)#, proxies=proxies)
        #search_result = r.content
        amazon_search_page_list=[]
        for page_no in range(start_page,end_page+1):
            #print(page_no)
            amazon_search_page = amazon_url+f"&page={page_no}"
            #uClient = uReq(amazon_search_page)
            #amazonPage= uClient.read()
            #print(amazon_search_page)
            amazon_search_page_list.append(amazon_search_page)
            #uClient.close()
            #print(amazon_search_page_list)
        #r.encoding='utf-8'
        #search_result = r.content
        #soup = bs(search_result,'html.parser')
       
        #uClient = uReq(amazon_url)
        #amazonPage= uClient.read()
        #uClient.close()
        
        all_review_page_list = []
        product_id_list = []
        product_name_list=[]
                
        for amazon_page_url in amazon_search_page_list:
            #print(amazon_page_url)
            r = requests.get(amazon_page_url, headers=headers)#, proxies=u[0])
            search_result = r.content
            #uClient = uReq(amazon_page_url)
            #amazonPage= uClient.read()
            #uClient.close()
            amazon_html = bs(search_result,'html.parser')
            #amazon_html;
            # Home Page for search results
            #bigboxes = amazon_html.findAll("div", {"class":"s-card-container s-overflow-hidden aok-relative puis-include-content-margin puis s-latency-cf-section s-card-border"})

            bigboxes = amazon_html.findAll('div', attrs={'data-component-type':'s-search-result'})
            #bigboxes;  
            #print(len(bigboxes))

            #session = Session()
            #session.headers= headers        
            #all_review_page_list = []
            #product_id_list = []
            #product_name_list=[]
            for box in bigboxes:
                #link_tag = box.find("a", attrs={'class':'a-link-normal s-no-outline'})
                try:
                    productLink = "https://www.amazon.in"+ box.div.div.div.div.a["href"]
                except:
                    productLink = "https://www.amazon.in"+ box.div.div.div.findAll('a')[0]['href']
                    #print(productLink)
                    
                #productLink = "https://www.amazon.in"+ box.div.div.div.div.a["href"]
                #print(productLink)
                index = productLink.find('dp/')
                see_all_review_page = productLink[:index] + "product-reviews" + productLink[index+2:]
                #print(see_all_review_page)
                all_review_page_list.append(see_all_review_page)
                #links_tag_list.append("https://www.amazon.in"+ box.div.div.div.div.a["href"])
                product_id_list.append(box.get("data-asin"))
                product_name_list.append(box.div.h2.span)
 
        return product_id_list, product_name_list, all_review_page_list

    ################# Get Reviews Data Frame #######################################################
    
    def get_reviews(all_review_page_list,product_id,product_name,review_page_number_range):

        start_page = review_page_number_range[0]
        end_page = review_page_number_range[1]
        #headers = { 
        #            "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
        #            "accept-encoding": "gzip, deflate, br", 
        #            "accept-language": "en", 
        #            "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36", 
        #        }
        headers = { 
                "accept": "text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.9", 
                "accept-encoding": "gzip, deflate, br", 
                "accept-language": "en-GB,en-US;q=0.9,en;q=0.8", 
                "user-agent": "Mozilla/5.0 (Linux; Android 6.0; Nexus 5 Build/MRA58N) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Mobile Safari/537.36", 
            }

        #session = Session()
        #session.headers= headers    
        customer_name_list = []
        rating_list = []
        review_title_list = []
        reviews_list = []
        product_ID_final = []
        product_name_final=[]
        #index_for_product_id = iter([i for i in range(len(all_review_page_list)+1)])
        id_idex = iter([i for i in range(len(product_id)+1)])
        name_idex = iter([i for i in range(len(product_name)+1)])
        for link in all_review_page_list:
            #print('productLink is: ',link) 
            #index_for_product_id = iter([i for i in range(len(all_review_page_list))])
            try:
                productID = product_id[next(id_idex)]
            except:
                productID = "NA"
            try:
                productName = product_name[next(name_idex)].text
            except:
                productName = "NA"
            

            for review_page_number in range(start_page,end_page+1):
                next_review_page_link = link + f"/ref=cm_cr_arp_d_paging_btm_2?ie=UTF8&pageNumber={review_page_number}&reviewerType=all_reviews"
                #print(next_review_page)    
                next_review_page = requests.get(next_review_page_link, headers=headers, timeout=None)
                next_review_page.encoding='utf-8'
                next_review_page_soup = bs(next_review_page.content, "html.parser")
                profiles = next_review_page_soup.findAll("span", {"class":"a-profile-name"})
                #print(next_review_page_link)
                for i in range(len(profiles)):
                    customer_name = profiles[i].text
                    Review_title = next_review_page_soup.findAll('span',{'class':"a-size-base review-title-content a-color-base a-text-bold",'data-hook':"review-title"})[i].text
                    reviews = next_review_page_soup.findAll("span", {"data-hook":"review-body"})
                    #print(reviews)
                    rating = next_review_page_soup.findAll("span", {"class":"a-icon-alt"})[i].text
                    #print(next_review_page_link)
                    #print(product_name)
                    #print(rating)
                    product_ID_final.append(productID)
                    product_name_final.append(productName)
                    #print(productName.text)
                    customer_name_list.append(customer_name)
                    rating_list.append(rating)
                   

                    if Review_title is not None:
                        try:
                            #print('try')
                            review_title_list.append(Review_title)
                        except:
                            #pass
                            review_title_list.append("NA")
                    else:
                        review_title_list.append("NA")
             
                    
                    if reviews is not None:
                        try:
                            #print('try')
                            reviews_list.append(reviews[i].text)
                        except:
                            #pass
                            reviews_list.append("NA")
                    else:
                        reviews_list.append("NA")
                    #time.sleep(random.randint(1,5))
                    
        df = pd.DataFrame([product_ID_final,product_name_final,customer_name_list, rating_list, review_title_list, reviews_list]).T
        df.columns = ["Product ID","ProductName",'Customer Name','Rating','Review Title','Reviews']
        df['Rating'] = df['Rating'].apply(lambda x: x.split()[0])
        df['Review Title'] = df['Review Title'].apply(lambda x: x.strip())
        df['Reviews'] = df['Reviews'].apply(lambda x: x.strip())
        df.to_csv('amazon_products_reviews.csv', index=False, encoding='utf-8') # Export Dataframe to csv
        return df


In [12]:
Amazon

__main__.Amazon

In [13]:
amazon_url = Amazon.search_product('jeans')

Retrying: Check Your Internet Connection: <urlopen error [Errno 11001] getaddrinfo failed>
Retrying: Check Your Internet Connection: <urlopen error [Errno 11001] getaddrinfo failed>
Retrying: Check Your Internet Connection: <urlopen error [Errno 11001] getaddrinfo failed>
Retrying: Check Your Internet Connection: <urlopen error [Errno 11001] getaddrinfo failed>
Retrying: Check Your Internet Connection: <urlopen error [Errno 11001] getaddrinfo failed>
Connection Established


In [14]:
amazon_url

'https://www.amazon.in/s?k=jeans'

In [6]:
Amazon.get_products_data(1,amazon_url)

([['World’s Best Inspirational Books to Change Your Life (Box Set of 3 Books)',
   '4.6 out of 5 stars',
   '(966)',
   '₹279',
   'NA',
   '₹599'],
  ['World’s Greatest Books For Personal Growth & Wealth (Set of 4 Books): Perfect Motivational Gift Set',
   '4.5 out of 5 stars',
   '(21,758)',
   '₹299',
   'NA',
   '₹699'],
  ['Do It Today: Overcome Procrastination, Improve Productivity',
   '4.5 out of 5 stars',
   '(3,491)',
   '₹98',
   'NA',
   '₹199'],
  ['The Power of Your Subconscious Mind',
   '4.5 out of 5 stars',
   '(64,403)',
   '₹105',
   'NA',
   '₹199'],
  ['Little Colouring Books for Kids (Set of 10 Books) - Gift to children for painting, drawing and colouring - Alphabets, Animals, Birds, Flags, Flowers, ... Transport, Vegetables - 3 to 6 years old',
   '4.3 out of 5 stars',
   '(1,897)',
   '₹195',
   'NA',
   '₹300'],
  ['You Can', '4.3 out of 5 stars', '(2,485)', '₹99', 'NA', '₹150'],
  ['Brain Activity Book for Kids - 200+ Activities for Age 3+',
   '4.5 out of 5 s

In [15]:

df= Amazon.generate_detailed_df([1,1],amazon_url)

Scrapping Page 1 of search results
Incomplete Scrapping: Few or Many Product May have been missed
We recommend you to retry after some time


In [16]:
df

,Product ID,Product Name,Rating,Customers Rated,Dicounted Price,Discount,Price
0,B0B7XLLY59,Dark Blue Jeans,3.5,2,"₹1,649",50%,3299
1,B0B7XRJFPQ,Medium Blue Jeans,NA,0,"₹1,249",50%,2499
2,B07L95H85P,Women's Black Skinny Fit High Rise Clean Look ...,4.0,"1,231",₹702,64%,1949
3,B085J7164M,Men Stretchable Jeans,3.8,"1,730",₹759,60%,1899
4,B09Z77V7F7,Men's 511 Styled Slim Fit Jeans,4.0,209,"₹1,739",40%,2899
5,B0B58RBN53,Men's Denim Slim Fit Strechable Jeans,3.5,260,₹629,79%,2999
6,B09Z77K2VS,Men's 511 Slim Fit Jeans,3.9,55,"₹1,739",40%,2899
7,B019XSHDL8,Men's Relaxed Jeans,3.6,"28,679",₹629,75%,2499
8,B085J74QM6,Men Jeans,3.7,"1,464",₹699,71%,2399
9,B07HZFHQ2D,Men Stretchable Jeans,3.6,927,₹759,70%,2499


In [ ]:
df['Product ID'].unique()

In [36]:
df.to_dict();

In [8]:
product_id, product_name, review_list = Amazon.get_products_review_link(amazon_url,[1,1])

In [54]:
len(review_list)
#review_list

39

In [102]:
index_for_product_id = iter([i for i in range(len(review_list))])
#product_id[next(index_for_product_id)]
product_id[next(index_for_product_id)]

'1472295935'

In [70]:
productName = product_name[next(index_for_product_id)].text
productName

"Life's Amazing Secrets: How to Find Balance and Purpose in Your Life | Inspirational Zen book on motivation, self-development & healthy living"

In [ ]:

for link in range(len(review_list)+1):
    print(product_id[next(index_for_product_id)])


In [9]:
df = Amazon.get_reviews(review_list,product_id,product_name,review_page_number_range=[1,1])
df

,Product ID,ProductName,Customer Name,Rating,Review Title,Reviews


In [14]:
len(df['Product ID'].unique())

47

In [13]:
len(df['ProductName'].unique())

44

# Reading CSV fie downloaded from webApp

In [5]:
import pandas as pd
#pd.read_csv('amazon (11).csv',error_bad_lines=False)
pd.read_csv('amazon (11).csv',on_bad_lines='skip')

,Unnamed: 0,Product ID,Product Name,Customer Name,Rating,Review Title,Reviews
0,0,B07H3GCFKP,F023 T-Shirt Padded Bra,Madhuri,3.8,Perfect fit ..no damage happened ..,Ordered two of these bras n packaging was ok t...
1,1,B07H3GCFKP,F023 T-Shirt Padded Bra,job seeker,4.0,Comfortable,Comfortable
2,2,B07H3GCFKP,F023 T-Shirt Padded Bra,payal,4.4,Not so good as picture,Fabric quality not so good stitching quality a...
3,6,B0B9JG5MWN,Women's Cotton Blend Lightly Padded Wirefree T...,Aishwarya,4.0,Very comfortable and soft.,Soft and comfortable.Fits perfectly to the size
4,7,B0B9JG5MWN,Women's Cotton Blend Lightly Padded Wirefree T...,Thallapaneni Nandini,4.0,Received different colours,Received different colours and size also not f...
...,...,...,...,...,...,...,...
118,177,B07KJT6KG4,Brand Non-Wired Fixed Straps Lightly Padded Wo...,Shwetha,4.0,Color mismatch,Comfort for use
119,178,B07KJT6KG4,Brand Non-Wired Fixed Straps Lightly Padded Wo...,ANKUR S.,4.0,Good product,Good and comfortable but size not fitted
120,179,B07KJT6KG4,Brand Non-Wired Fixed Straps Lightly Padded Wo...,N,5.0,Product Packing bahut hi बकवास,Good and comfortable but size not fitted
121,182,B07KJT6KG4,Brand Non-Wired Fixed Straps Lightly Padded Wo...,Amazon Customer,5.0,Good quality product,Good genuine product.


# Proxy

In [279]:
from bs4 import BeautifulSoup
import requests
country_code="in"
l={}
u=list()
url="https://www.proxynova.com/proxy-server-list/country-"+country_code+"/"
respo = requests.get(url).text
soup = BeautifulSoup(respo,"html.parser")
allproxy = soup.find_all("tr")
for proxy in allproxy:
    foo = proxy.find_all("td")
    try:
        l["ip"]=foo[0].text.replace("\n","").replace("document.write(","").replace(")","").replace("\'","").replace(";","")
        
    except:
        l["ip"]=None
    try:
        l["port"]=foo[1].text.replace("\n","").replace(" ","")
        #print(l["port"])
    except:
        l["port"]=None
    try:
        l["country"]=foo[5].text.replace("\n","").replace(" ","")
    except:
        l["country"]=None
    if(l["port"] is not None):
        u.append(l)
        
        l={}
        #print(u)

In [280]:
dfp =  pd.DataFrame(u)


In [281]:
dfp['ip']= ["103.251.214.167", "103.79.160.0",
 "134.238.252.143", "103.125.154.233",
 "103.155.54.185", "103.123.234.106",
 "143.244.133.78", "103.42.162.50",
 "115.242.198.158", "14.140.131.82",
"202.131.159.210", "115.247.13.154", "115.243.88.49",
 "13.126.92.209", "202.91.77.222",
 "103.148.39.42", "103.155.54.233",
 "43.243.172.2", "3.109.85.109" ,"103.48.68.37",
"103.48.68.34", "43.204.16.38",
 "103.172.179.226","117.240.206.222","15.207.51.7",
 "103.248.120.5","103.242.119.88",
 "143.110.248.3","38.9.58.105",
 "206.189.136.143","202.91.70.202","49.248.32.11",
 "103.48.68.36","115.243.88.51","103.142.108.153"
 ]

In [282]:
dfp

,ip,port,country
0,103.251.214.167,80,India-Mumbai
1,103.79.160.0,8080,India-Mumbai
2,134.238.252.143,3128,India-Mumbai
3,103.125.154.233,80,India-Mumbai
4,103.155.54.185,83,India-Mumbai
5,103.123.234.106,80,India-Mumbai
6,143.244.133.78,83,India-Mumbai
7,103.42.162.50,8080,India-AnandParbat
8,115.242.198.158,8080,India-Mumbai
9,14.140.131.82,8080,India-Mumbai


In [283]:

#dfp['ip']+dfp['port']

dfp = dfp[dfp['port']=='8080' ]
dfp

,ip,port,country
1,103.79.160.0,8080,India-Mumbai
7,103.42.162.50,8080,India-AnandParbat
8,115.242.198.158,8080,India-Mumbai
9,14.140.131.82,8080,India-Mumbai
10,202.131.159.210,8080,India-Alappuzha
12,115.243.88.49,8080,India
16,103.155.54.233,8080,India-Thrissur
18,3.109.85.109,8080,India-Kilakarai
19,103.48.68.37,8080,India-Mumbai
25,103.248.120.5,8080,India


In [292]:
diction = []
for i in range(dfp.shape[0]):
    a=dfp.iloc[i]['ip']+":" + dfp.iloc[i]['port']
    diction.append({"http":a})

In [299]:
for i in diction:
    print(i)

{'http': '103.79.160.0:8080'}
{'http': '103.42.162.50:8080'}
{'http': '115.242.198.158:8080'}
{'http': '14.140.131.82:8080'}
{'http': '202.131.159.210:8080'}
{'http': '115.243.88.49:8080'}
{'http': '103.155.54.233:8080'}
{'http': '3.109.85.109:8080'}
{'http': '103.48.68.37:8080'}
{'http': '103.248.120.5:8080'}
{'http': '202.91.70.202:8080'}


In [ ]:
diction